# **Spring2024_Simple_Data_Migration_from_RethinkDB_to_MariaDB Progress Report 1** 
April 26th

## Description: 
Migrate data from RethinkDB to MariaDB using Python scripts where data stored in a RethinkDB database is transferred to a MariaDB database. Develop Python scripts to retrieve data from RethinkDB and insert it into MariaDB, ensuring data integrity and consistency. Utilize the rethinkdb and mysql-connector-python libraries for database interaction, showcasing basic data migration techniques between different database systems. Add different functionalities to enhance complexity of project.

Link: https://docs.google.com/document/d/1GEOmfpBUXiCua18wR1Hx1OMUVlku-1of/edit#heading=h.4ph7eurcozb1

## About This Project:
Everything below are my personal understanding, please feel free to correct me if you see any mistakes, any comment is appreciaated, thank you!

It is my understanding, this project contains the following step:
### 0. Import needed Libraries
### 1. Establish connection to the databases
### 2. Listed Data
### 3. Image Data
### 4. Audio Data
### 5. Video Data

## 0. Import needed Libraries


In [1]:
from moviepy.editor import VideoFileClip
import cv2
import numpy as np
import time
from scipy.io import savemat
from scipy.io.wavfile import read, write
import rethinkdb as r
import json
import pandas as pd
import mariadb
import sys
from pydub import AudioSegment
import soundfile as sf
import json

def split(array):
    array = np.array(array)
    red = array[:, :, 0:1]
    green = array[:, :, 1:2]
    blue = array[:, :, 2:3]
    nb = []
    nr = []
    ng = []
    for i in red:
        tp = [item for sublist in i for item in sublist]
        tp = [int(x) for x in tp]
        nr.append(tp)
        i = None
    red = None
    for j in green:
        tp = [item for sublist in j for item in sublist]
        tp = [int(x) for x in tp]
        ng.append(tp)
        j = None
    green = None
    for k in blue:
        tp = [item for sublist in k for item in sublist]
        tp = [int(x) for x in tp]
        nb.append(tp)
        k = None
    blue = None
    red = None
    green = None
    return nr, ng, nb


In [2]:
#This block is used to test that everything is running, it doesn't serve any purpous
cap = cv2.VideoCapture('./Rickroll.mp4')
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("there are", length, "frames in the chosen video")

there are 0 frames in the chosen video


## 1. establish the connections:
here I connect this notebook to a rethinkdb database running locally, I think the ultimate goal is to access the data from a remote database, but I am still working out the kinks of that...So for now, I will be working with a locally hosted Rethindb Database. 

In [3]:
rethink = r.RethinkDB()
rethink.connect('rethinkdb', 28015).repl()

As you might have noticed, I have an image of earth named earth.jpg in my code file, that is because at the begining, I wasn't able to store the entire video in the rethinkdb database, so I admitted defeat and was going to migrate a picture instead of an video. But I later (around the first woking in class session) found a way to store the video in a rethinkdb database, so the following code where I stores the picture is no longer needed. 

In [4]:
try:
    # connection parameters
    conn_params = {
        'user' : "root",
        'password' : "<196900>",
        'host' : "172.18.0.1",
        'port' : 3306,
        'database' : "<data>"
    }

    # establish a connection
    connection = mariadb.connect(**conn_params)
    cursor = connection.cursor()
    
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)
    
print(cursor)

## 3. Listed Data


In [5]:
rethink.db_create("listed").run()#Create database in rethink for this section
rethink.db("listed").table_create("data").run()#create table for this section

{'config_changes': [{'new_val': {'db': 'listed',
    'durability': 'hard',
    'id': '577dd6cc-7cfc-40a0-8fb5-b801fa507068',
    'indexes': [],
    'name': 'data',
    'primary_key': 'id',
    'shards': [{'nonvoting_replicas': [],
      'primary_replica': 'c628dc9d9e94_x94',
      'replicas': ['c628dc9d9e94_x94']}],
    'write_acks': 'majority',
    'write_hook': None},
   'old_val': None}],
 'tables_created': 1}

In [6]:
df = pd.read_csv('listeddata.csv')#Read in the csv file
listed_column = df.columns.tolist()#Create a Dataframe of the data
print(len(listed_column))#learn about the dataframe
tp = df
tp.astype(str)#Convert all member of the frame into string
for index, line in tp.iterrows():#Prepare the data fore importing to rethinkdb
    x = list(line)
    converted = [str(i) for i in x]
    dict_data = {listed_column[i]:converted[i] for i in range(len(x))}#This turns the data into dictionaries so that they will be inserted correcly
    dict_data['order']=index#Add another member to the current dictionary to track its order so that we can recreate it
    #print(dict_data)
    rethink.db('listed').table('data').insert(dict_data).run()#inser data into rethinkdb
    print("line", index, "inserted successfully!")

2
line 0 inserted successfully!
line 1 inserted successfully!
line 2 inserted successfully!
line 3 inserted successfully!
line 4 inserted successfully!
line 5 inserted successfully!
line 6 inserted successfully!
line 7 inserted successfully!
line 8 inserted successfully!
line 9 inserted successfully!
line 10 inserted successfully!
line 11 inserted successfully!
line 12 inserted successfully!
line 13 inserted successfully!
line 14 inserted successfully!
line 15 inserted successfully!
line 16 inserted successfully!
line 17 inserted successfully!
line 18 inserted successfully!
line 19 inserted successfully!
line 20 inserted successfully!
line 21 inserted successfully!
line 22 inserted successfully!
line 23 inserted successfully!
line 24 inserted successfully!
line 25 inserted successfully!
line 26 inserted successfully!
line 27 inserted successfully!
line 28 inserted successfully!
line 29 inserted successfully!
line 30 inserted successfully!
line 31 inserted successfully!
line 32 inserted

In [7]:
print(dict_data.keys())#get a list of columns in rethinkdb

dict_keys(['Date', 'TouristNumber', 'order'])


In [8]:
#retrieve data according to key
fun = rethink.db('listed').table('data').filter({'TouristNumber':'8414'}).run()
tr = list(fun)
print(type(fun))
print(tr)

<class 'rethinkdb.net.DefaultCursor'>
[{'Date': '33604', 'TouristNumber': '8414', 'id': 'b4674a4b-c0ec-4c12-81f0-7e366c182a8f', 'order': 0}]


In [9]:
fun = rethink.db('listed').table('data').filter({'Date':'33756'}).run()
tr = list(fun)
print(type(fun))
print(tr)

<class 'rethinkdb.net.DefaultCursor'>
[{'Date': '33756', 'TouristNumber': '46383', 'id': 'df23acb7-60d7-458b-8601-f2037f279b72', 'order': 5}]


In [10]:
fun = rethink.db('listed').table('data').filter({'order':44}).run()
tr = list(fun)
print(type(fun))
print(tr)

<class 'rethinkdb.net.DefaultCursor'>
[{'Date': '34943', 'TouristNumber': '27665', 'id': '37181e38-ccfe-42ff-b732-7fe6318d107d', 'order': 44}]


In [11]:
cursor.execute("CREATE TABLE listed_data (date int, tournum int, od int ,id TEXT)")#Create Mariadb table for the project

In [12]:
stored = rethink.db('listed').table('data').run()#pull all the data from rethinkdb
i = 1
for document in stored:#iterate through pulled data
    date = document['Date']
    tnum = document['TouristNumber']
    id = document['id']
    order = document['order']
    query = f"INSERT INTO listed_data (date, tournum, od ,id ) VALUES ('{date}', '{tnum}', '{order}', '{id}')" #import the data into mariadb
    cursor.execute(query) 
    connection.commit()

In [13]:
cursor.execute("SELECT date, tournum, od FROM listed_data")#Retrieve data from mariadb to test for accuracy
tblue = cursor.fetchall()
print(tblue[2])

(41760, 47387, 268)


In [14]:
recreate = pd.DataFrame(tblue, columns = dict_data.keys())#reform the data into its origional states
sor = recreate.sort_values(by='order')
sor = sor.drop(columns=['order'])
sor = sor.reset_index(drop = True)
#sor = sor.drop(columns=['index'])
print(sor.head())
print(tp.head())
print(tp.equals(sor))

    Date  TouristNumber
0  33604           8414
1  33635           9767
2  33664          13805
3  33695          12987
4  33725          32190
    Date  TouristNumber
0  33604           8414
1  33635           9767
2  33664          13805
3  33695          12987
4  33725          32190
True


## 4. Audio Data


In [15]:
adata, samplerate = sf.read('music.mp3')#read in the audio data
dim = adata.shape#get a first idea of the dataset
print(type(adata))
#print(type(adata[12][1]))
ladata = adata.tolist()#turn data into lists they were arrays
print(type(ladata[2]))
num_arrays = len(adata) // samplerate#find out how many seconds the audio lasts for
smaller_arrays = [adata[i*samplerate:(i+1)*samplerate] for i in range(num_arrays)]#store the data in terms of second, each second is a row
smaller_arrays.append(adata[num_arrays*samplerate:])#make sure all the data is included
print(dim)#tests
print(len(smaller_arrays))#tests
print(len(smaller_arrays[1]))#tests
print(len(adata))#tests

<class 'numpy.ndarray'>
<class 'list'>
(8017920, 2)
168
48000
8017920


In [16]:
rethink.db_create("Audio").run()#create rethinkdb database
rethink.db("Audio").table_create("ladata").run()#create rethinkdb table

{'config_changes': [{'new_val': {'db': 'Audio',
    'durability': 'hard',
    'id': 'e8a5a3aa-a92e-4942-a23f-6dd4fccfcb38',
    'indexes': [],
    'name': 'ladata',
    'primary_key': 'id',
    'shards': [{'nonvoting_replicas': [],
      'primary_replica': 'c628dc9d9e94_x94',
      'replicas': ['c628dc9d9e94_x94']}],
    'write_acks': 'majority',
    'write_hook': None},
   'old_val': None}],
 'tables_created': 1}

In [17]:
for i in range(len(smaller_arrays)):#flatten the data and import it into rethindb
    smaller_arrays[i] = smaller_arrays[i].reshape(1, -1)
    rethink.db('Audio').table('ladata').insert({'data':smaller_arrays[i], 'order':str(i)}).run()#order was there to make sure we can recreate our data
    print(i, 'has been inserted')    

0 has been inserted
1 has been inserted
2 has been inserted
3 has been inserted
4 has been inserted
5 has been inserted
6 has been inserted
7 has been inserted
8 has been inserted
9 has been inserted
10 has been inserted
11 has been inserted
12 has been inserted
13 has been inserted
14 has been inserted
15 has been inserted
16 has been inserted
17 has been inserted
18 has been inserted
19 has been inserted
20 has been inserted
21 has been inserted
22 has been inserted
23 has been inserted
24 has been inserted
25 has been inserted
26 has been inserted
27 has been inserted
28 has been inserted
29 has been inserted
30 has been inserted
31 has been inserted
32 has been inserted
33 has been inserted
34 has been inserted
35 has been inserted
36 has been inserted
37 has been inserted
38 has been inserted
39 has been inserted
40 has been inserted
41 has been inserted
42 has been inserted
43 has been inserted
44 has been inserted
45 has been inserted
46 has been inserted
47 has been inserted
48

In [18]:
second = rethink.db('Audio').table('ladata').filter({'order':'1'}).run()#this return the data of a certain second
audio_for_second = list(second)
print(type(audio_for_second[0]))

<class 'dict'>


In [19]:
x=1#this gives the data to create a 10s teaser, x will be the begining second(1 minute and 1 second should be 61 instead of 1.1)
teaser = []
for i in range(10):
    second = rethink.db('Audio').table('ladata').filter({'order':str(x+i)}).run()
    teaser.append(second) 
    print(x+i, 'seconds data has been loaded')

1 seconds data has been loaded
2 seconds data has been loaded
3 seconds data has been loaded
4 seconds data has been loaded
5 seconds data has been loaded
6 seconds data has been loaded
7 seconds data has been loaded
8 seconds data has been loaded
9 seconds data has been loaded
10 seconds data has been loaded


In [20]:
cursor.execute("CREATE TABLE audios (data MEDIUMTEXT, ord int)")#Create the table for this section

In [21]:
audio = rethink.db('Audio').table('ladata').run()#retrieve the data from rethinkdb
for document in audio:
    second = document['data']
    od = int(document['order'])
    query = f"INSERT INTO audios (data, ord) VALUES ('{second}', '{od}')" #import the data into mariadb with od to keep track of order for recreation
    cursor.execute(query) 
    connection.commit() 
    print(od)

92
115
60
155
94
52
158
6
8
134
84
102
80
126
69
81
41
119
79
107
99
161
128
9
110
1
127
139
101
57
5
136
104
166
3
163
49
74
58
138
24
142
91
89
10
152
72
93
86
117
35
96
7
53
100
59
124
56
27
106
55
150
145
76
82
114
42
39
62
95
140
113
154
34
83
67
160
71
26
21
78
137
167
88
165
0
30
45
131
63
135
77
23
164
157
54
111
17
20
125
122
132
90
48
97
19
143
85
43
36
33
146
144
153
28
116
105
12
50
15
87
70
109
68
31
159
14
103
44
118
120
51
108
156
133
98
13
37
64
130
121
32
151
123
61
40
73
75
25
18
47
22
11
147
65
129
149
16
141
112
46
2
4
148
29
162
38
66


In [22]:
cursor.execute("SELECT data, ord FROM audios")#pull data from mariadb
tblue = cursor.fetchall()
readata = []
print(type(tblue[1]))
sortedata = sorted(tblue, key=lambda x: x[1])#recreate the data so that we can compare
i = 1
for i in sortedata: #iterate through retrieved data to reform them
    reformed_array= np.array(eval(i[0]))
    readata.append(reformed_array[0].reshape((-1, 2)))
    print('done')
sortedata = []
print(len(readata))

<class 'tuple'>
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
168


In [23]:
print(readata[1].shape)

(48000, 2)


In [24]:
final = np.vstack(readata)#recreation and check for accuracy
print(final.shape)
are_equal = np.array_equal(final, adata)

print(are_equal)

(8017920, 2)
True


## 5. Image Data


In [25]:
# Read the image using OpenCV
image = cv2.imread('Small_Earth.jpg')

# OpenCV reads images in BGR format by default
# Convert it to RGB format

print(image.shape)

(410, 410, 3)


In [26]:
rethink.db_create("Image").run()#create the database
rethink.db("Image").table_create("idata").run()#create the table

{'config_changes': [{'new_val': {'db': 'Image',
    'durability': 'hard',
    'id': '1915ac8d-0667-4b76-b7b1-93dbbf5cc013',
    'indexes': [],
    'name': 'idata',
    'primary_key': 'id',
    'shards': [{'nonvoting_replicas': [],
      'primary_replica': 'c628dc9d9e94_x94',
      'replicas': ['c628dc9d9e94_x94']}],
    'write_acks': 'majority',
    'write_hook': None},
   'old_val': None}],
 'tables_created': 1}

In [27]:
B = image[:,:,0].reshape(2, -1).astype(int)#prepare the data so that rethink will accept them
G = image[:,:,1].reshape(2, -1).astype(int)
R = image[:,:,2].reshape(2, -1).astype(int)

Rs= R.tolist()
Gs = G.tolist()
Bs = B.tolist()
data = {"red":Rs, "green":Gs, "blue":Bs}
rethink.db("Image").table("idata").insert(data).run()#inser the data into rethinkdb

{'deleted': 0,
 'errors': 0,
 'generated_keys': ['a7881164-86a0-4408-a090-c9ffacecee70'],
 'inserted': 1,
 'replaced': 0,
 'skipped': 0,
 'unchanged': 0}

In [28]:
cursor.execute("CREATE TABLE image (R MEDIUMTEXT,G MEDIUMTEXT, B MEDIUMTEXT)")#Create the table in Mariadb for this section

In [29]:
img = rethink.db("Image").table("idata").run()#get data from rethinkdb
for document in img:
    colR = document['red']
    colG = document['green']
    colB = document['blue']
# Combine the two lists
colR = colR[0] + colR[1]
colG = colG[0] + colG[1]
colB = colB[0] + colB[1]
scolR = [str(item) for item in colR]
scolG = [str(item) for item in colG]
scolB = [str(item) for item in colB]
Rfin = ','.join(scolR)
Gfin = ','.join(scolG)
Bfin = ','.join(scolB)
query = f"INSERT INTO image (R, G, B) VALUES ('{Rfin}', '{Gfin}', '{Bfin}')" #insert data into mariadb
cursor.execute(query) 
connection.commit()

In [30]:
cursor.execute("SELECT R, G, B FROM image")#retrieve data from mariadb
tblue = cursor.fetchall()
R = tblue[0][0].split(',')
G = tblue[0][1].split(',')
B = tblue[0][2].split(',')
R = [int(x) for x in R]
G = [int(x) for x in G]
B = [int(x) for x in B]
R = np.array(R).reshape(410, 410)
G = np.array(G).reshape(410, 410)
B = np.array(B).reshape(410, 410)
BGR_array = np.stack((B, G, R), axis=-1)
are_equal = np.array_equal(image, BGR_array)
print(are_equal)#check for accuracy

True


## 6. Video Data

In [31]:
cap = cv2.VideoCapture('videoplayback.mp4')#read the video into here
audios = VideoFileClip('videoplayback.mp4').audio
#audios.write_audiofile('audio.wav')
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(length)#check the number of frames in that video

266


In [31]:
rethink.db_create("film").run()#create the database
rethink.db("film").table_create("video").run()
rethink.db("film").table_create("audio").run()#Create the table for this section

{'config_changes': [{'new_val': {'db': 'film',
    'durability': 'hard',
    'id': 'c2e8b75d-6fdf-486d-a56c-67b86f4693dd',
    'indexes': [],
    'name': 'audio',
    'primary_key': 'id',
    'shards': [{'nonvoting_replicas': [],
      'primary_replica': '09d46aa8fe1b_nfs',
      'replicas': ['09d46aa8fe1b_nfs']}],
    'write_acks': 'majority',
    'write_hook': None},
   'old_val': None}],
 'tables_created': 1}

In [32]:
for i in range(length):#prepare the data for rethinkdb, This take a long time to do...
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()
    frame = frame.astype(str)
    ri = frame[:,:,0].reshape(3, -1)
    bi = frame[:,:,1].reshape(3, -1)
    gi = frame[:,:,2].reshape(3, -1)
#testr = ri.reshape(1,-1)
#tesg = gi.reshape(1,-1)
#testb = bi.reshape(1,-1)
#testr = testr.tolist()
    list_r= [list(array) for array in ri]
    list_g= [list(array) for array in gi]
    list_b= [list(array) for array in bi]
    data = {"id":i,"red":list_r, "green":list_g, "blue":list_b}#inser the data into rethinkdb
    rethink.db("film").table("video").insert(data).run()
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265


In [33]:
second = rethink.db('film').table('video').filter({'id':15}).run()#get the data from mariadb
audio_for_second = list(second)
print(type(audio_for_second[0]))

<class 'dict'>


In [34]:
x=1#same as audio, but this time it gets video data
teaser = []
for i in range(10):#retrieve the 10 frame of data
    second = rethink.db('film').table('video').filter({'id':x+i}).run()
    tp = list(second)
    teaser.append(tp) 
    print(x+i, 'seconds data has been loaded')
    print(type(tp[0]))

1 seconds data has been loaded
<class 'dict'>
2 seconds data has been loaded
<class 'dict'>
3 seconds data has been loaded
<class 'dict'>
4 seconds data has been loaded
<class 'dict'>
5 seconds data has been loaded
<class 'dict'>
6 seconds data has been loaded
<class 'dict'>
7 seconds data has been loaded
<class 'dict'>
8 seconds data has been loaded
<class 'dict'>
9 seconds data has been loaded
<class 'dict'>
10 seconds data has been loaded
<class 'dict'>


In [35]:
cursor.execute("CREATE TABLE videos (R MEDIUMTEXT, G MEDIUMTEXT, B MEDIUMTEXT, id int)")#create the table

In [36]:
dt = []
tr = rethink.db('film').table('video').run()#retrieve data from rethinkdb and get them ready for mariadb
for doc in tr:
    colR = doc['red']
    colG = doc['green']
    colB = doc['blue']
    od = doc['id']
# Combine the two lists
    colR = colR[0] + colR[1] + colR[2]# combine the 3 lists
    colG = colG[0] + colG[1] + colG[2]
    colB = colB[0] + colB[1] + colB[2]
    scolR = [str(item) for item in colR]# turn them into strings for mariadb
    scolG = [str(item) for item in colG]
    scolB = [str(item) for item in colB]
    Rfin = ','.join(scolR)#turn the whole list into one string
    Gfin = ','.join(scolG)
    Bfin = ','.join(scolB)
    query = f"INSERT INTO videos (R, G, B, id) VALUES ('{Rfin}', '{Gfin}', '{Bfin}', '{od}')"#insert data into mariadb
    cursor.execute(query) 
    connection.commit()
    print("done")

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


In [37]:
cursor.execute("SELECT R, G, B , id FROM videos")#retrieve data from mariadb
tblue = cursor.fetchall()
print(type(tblue[0][3]))
sorted_tblue = sorted(tblue, key=lambda x: x[3])#order them correctly
print(len(tblue))

<class 'int'>
266


In [38]:
ori_frames = []#recreate the data and compare for accuracy
for i in range(length):
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()
    ori_frames.append(frame)
print(len(ori_frames))
for i in range(len(sorted_tblue)):
    R = sorted_tblue[i][0].split(',')
    G = sorted_tblue[i][1].split(',')
    B = sorted_tblue[i][2].split(',')
    R = [int(x) for x in R]
    G = [int(x) for x in G]
    B = [int(x) for x in B]
    R = np.array(R).reshape(360, 640)
    G = np.array(G).reshape(360, 640)
    B = np.array(B).reshape(360, 640)
    BGR_array = np.stack((G, B, R), axis=-1)
    are_equal = np.array_equal(ori_frames[i], BGR_array)
    print(are_equal)


266
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [34]:
print(frame.shape)

(360, 640, 3)
